In [21]:
import numpy as np
import pandas as pd
import os
import torch
import torchaudio
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import Wav2Vec2Processor, Wav2Vec2Model

In [22]:
# 모델 준비
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

C:\Users\asia\.conda\envs\py312tf9\Lib\site-packages\transformers\configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Wav2Vec2Encoder(
    (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
  

In [23]:
df = pd.read_csv("aihub_emotion_preprocessed_cleaned.csv")
df = df[df["final_emotion"] != "disgust"]

audio_dirs = ['./감정 분류를 위한 대화 음성 데이터셋/4차년도', './감정 분류를 위한 대화 음성 데이터셋/5차년도', './감정 분류를 위한 대화 음성 데이터셋/5차년도_2차']
def find_existing_audio_path(wav_id):
    for d in audio_dirs:
        path = os.path.join(d, wav_id + ".wav")
        if os.path.exists(path):
            return path
    return None  # 어떤 폴더에도 없으면 None
df["file_path"] = df["wav_id"].apply(find_existing_audio_path)


In [24]:
df = df[df["file_exist"] == True].reset_index(drop=True)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13921 entries, 0 to 13920
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   wav_id         13921 non-null  object
 1   발화문            13921 non-null  object
 2   final_emotion  13921 non-null  object
 3   1번 감정          13921 non-null  object
 4   1번 감정세기        13921 non-null  int64 
 5   2번 감정          13921 non-null  object
 6   2번 감정세기        13921 non-null  int64 
 7   3번 감정          13921 non-null  object
 8   3번 감정세기        13921 non-null  int64 
 9   4번 감정          13921 non-null  object
 10  4번감정세기         13921 non-null  int64 
 11  5번 감정          13921 non-null  object
 12  5번 감정세기        13921 non-null  int64 
 13  나이             13921 non-null  int64 
 14  성별             13921 non-null  object
 15  source         13921 non-null  object
 16  wav_path       13921 non-null  object
 17  file_exist     13921 non-null  bool  
 18  file_path      13921 non-n

In [26]:
emotion_counts = df["final_emotion"].value_counts()
emotion_ratio = df["final_emotion"].value_counts(normalize=True) * 100

# 감정별 샘플 수와 비율 함께 출력
emotion_summary = pd.DataFrame({
    "count": emotion_counts,
    "ratio (%)": emotion_ratio.round(2)
})

print(emotion_summary)

               count  ratio (%)
final_emotion                  
anger           8370      60.12
happiness       2555      18.35
neutral         1529      10.98
sadness         1467      10.54


In [27]:
df.head()

,wav_id,발화문,final_emotion,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별,source,wav_path,file_exist,file_path
0,5e258fd1305bcf3ad153a6a4,"어, 청소 니가 대신 해 줘!",anger,Neutral,0,Angry,1,Neutral,0,Neutral,0,Angry,1,27,male,./4차년도,감정 분류를 위한 대화 음성 데이터셋\./4차년도\5e258fd1305bcf3ad1...,True,./감정 분류를 위한 대화 음성 데이터셋/4차년도\5e258fd1305bcf3ad1...
1,5e258fe2305bcf3ad153a6a5,둘 다 청소 하기 싫어. 귀찮아.,anger,Neutral,0,Angry,1,Neutral,0,Neutral,0,Angry,1,27,male,./4차년도,감정 분류를 위한 대화 음성 데이터셋\./4차년도\5e258fe2305bcf3ad1...,True,./감정 분류를 위한 대화 음성 데이터셋/4차년도\5e258fe2305bcf3ad1...
2,5e258ff5305bcf3ad153a6a6,둘 다 하기 싫어서 화내.,anger,Angry,1,Angry,1,Neutral,0,Angry,1,Angry,1,27,male,./4차년도,감정 분류를 위한 대화 음성 데이터셋\./4차년도\5e258ff5305bcf3ad1...,True,./감정 분류를 위한 대화 음성 데이터셋/4차년도\5e258ff5305bcf3ad1...
3,5e25902f305bcf3ad153a6a9,그럼 방세는 어떡해.,anger,Sadness,1,Sadness,1,Sadness,1,Sadness,1,Sadness,1,27,male,./4차년도,감정 분류를 위한 대화 음성 데이터셋\./4차년도\5e25902f305bcf3ad1...,True,./감정 분류를 위한 대화 음성 데이터셋/4차년도\5e25902f305bcf3ad1...
4,5e27fb575807b852d9e01595,어. 고등학교 동창인데 이렇게 더럽게 쓸줄 몰랐어.,anger,Disgust,2,Disgust,1,Angry,1,Disgust,2,Angry,1,32,male,./4차년도,감정 분류를 위한 대화 음성 데이터셋\./4차년도\5e27fb575807b852d9...,True,./감정 분류를 위한 대화 음성 데이터셋/4차년도\5e27fb575807b852d9...


In [28]:


def load_audio(file_path):
    waveform, sr = torchaudio.load(file_path)
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
        waveform = resampler(waveform)
    return waveform.squeeze()

In [29]:
def extract_embedding(wav_path):
    try:
        waveform = load_audio(wav_path)
        inputs = processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True)
        with torch.no_grad():
            outputs = model(**inputs.to(device))
        # [CLS] 벡터 대신 평균 사용
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
        return embedding
    except Exception as e:
        print(f"❌ 오류: {wav_path} — {e}")
        return None

In [30]:
embeddings = []
labels = []

for i, row in tqdm(df.iterrows(), total=len(df)):
    emb = extract_embedding(row["wav_path"])
    if emb is not None:
        embeddings.append(emb)
        labels.append(row["final_emotion"])

# numpy로 저장
X = np.stack(embeddings)
y = np.array(labels)

np.save("emotion_X_excl_disgust.npy", X)
np.save("emotion_y_excl_disgust.npy", y)

print(f"✅ 저장 완료: {X.shape}, {len(y)}")

100%|██████████| 13921/13921 [1:04:57<00:00,  3.57it/s]


✅ 저장 완료: (13921, 768), 13921


In [32]:
import librosa
import soundfile as sf

# 스테레오를 모노로 변환하여 저장
stereo_path = "./user_reference/user.wav"
mono_path = "./user_reference/user_mono.wav"

y, sr = librosa.load(stereo_path, sr=16000, mono=True)
sf.write(mono_path, y, sr)

# 임베딩 추출 및 저장
user_emb = extract_embedding(mono_path)
np.save("registered_user_embedding.npy", user_emb)
print("✅ 사용자 임베딩 저장 완료: registered_user_embedding.npy")

✅ 사용자 임베딩 저장 완료: registered_user_embedding.npy
